In [1]:
import os
import sys

# Allow imports from src folder
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

import betfairlightweight
from betfairlightweight import filters
from datetime import datetime, timedelta
from dateutil import tz
from pandas.tseries.offsets import MonthEnd

import math
import numpy as np
import pandas as pd
import fasttrack_dataset

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize

from matplotlib import pyplot
from matplotlib.pyplot import figure

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Import FastTrack dataset
race_details, dog_results = fasttrack_dataset.load(exclude_states=['NZ'])

Valid Security Key
Loading data from 2017-12-01 to 2017-12-31
Loading data from 2018-01-01 to 2018-01-31
Loading data from 2018-02-01 to 2018-02-28
Loading data from 2018-03-01 to 2018-03-31
Loading data from 2018-04-01 to 2018-04-30
Loading data from 2018-05-01 to 2018-05-31


/home/bruno/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Loading data from 2018-06-01 to 2018-06-30
Loading data from 2018-07-01 to 2018-07-31
Loading data from 2018-08-01 to 2018-08-31
Loading data from 2018-09-01 to 2018-09-30
Loading data from 2018-10-01 to 2018-10-31
Loading data from 2018-11-01 to 2018-11-30
Loading data from 2018-12-01 to 2018-12-31
Loading data from 2019-01-01 to 2019-01-31
Loading data from 2019-02-01 to 2019-02-28
Loading data from 2019-03-01 to 2019-03-31
Loading data from 2019-04-01 to 2019-04-30
Loading data from 2019-05-01 to 2019-05-31
Loading data from 2019-06-01 to 2019-06-30
Loading data from 2019-07-01 to 2019-07-31
Loading data from 2019-08-01 to 2019-08-31
Loading data from 2019-09-01 to 2019-09-30
Loading data from 2019-10-01 to 2019-10-31
Loading data from 2019-11-01 to 2019-11-30
Loading data from 2019-12-01 to 2019-12-31
Loading data from 2020-01-01 to 2020-01-31
Loading data from 2020-02-01 to 2020-02-29
Loading data from 2020-03-01 to 2020-03-31
Loading data from 2020-04-01 to 2020-04-30
Loading dat

/home/bruno/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Loading data from 2021-06-01 to 2021-06-30
Loading data from 2021-07-01 to 2021-07-31


,@id,RaceNum,RaceName,RaceTime,Distance,RaceGrade,Track,date
0,278896185,1,TRIPLE M BENDIGO 93.5,02:54PM,425m,Grade 6,Bendigo,01 Dec 17
1,278896189,2,GOLDEN CITY CONCRETE PUMPING,03:17PM,500m,Mixed 6/7,Bendigo,01 Dec 17
2,275589809,3,RAILWAY STATION HOTEL FINAL,03:38PM,500m,Mixed 6/7 Final,Bendigo,01 Dec 17
3,278896183,4,MCIVOR RD VETERINARY CLINIC,03:58PM,425m,Grade 5,Bendigo,01 Dec 17
4,278896179,5,GRV VIC BRED SERIES HT1,04:24PM,425m,Grade 5 Heat,Bendigo,01 Dec 17
